In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

from models.segvol.base import SegVol, SegVolModel, SegVolConfig
device = "cpu"

img_path = "../data/segthor_train/train/Patient_05/Patient_05.nii.gz"
gt_path = "../data/segthor_train/train/Patient_05/GT.nii.gz"

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2024-10-05 18:21:35.179792: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
config = SegVolConfig()

from models.segvol.base import SegVolProcessor

# NOTE: The spatial size is 32x256x256 which is not that much to capture the whole volume
processor = SegVolProcessor(config.spatial_size)


monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [3]:
ct, gt = processor.preprocess_ct_gt(img_path, gt_path, K=5)

# Add batch dimensions (don't ask me why I'm doing this, the transform function complains otherwise)
ct.unsqueeze_(0) # (1, 5, 32, 256, 256)


tensor([[[[-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          ...,
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880]],

         [[-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          ...,
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880]],

         [[-2.0880, -2.0880, -2.0880,  ..., -2.0880, -2.0880, -2.0880],
          [-2.0880, -2.0880, -

In [5]:
data_item = processor.train_transform(ct, gt)
image, gt3D = data_item["image"].to(device), data_item["label"].unsqueeze(0).to(device) # add batch dim

In [6]:
from models.segvol.base import TextEncoder
from transformers import AutoModel
model = AutoModel.from_pretrained(
            "BAAI/SegVol", trust_remote_code=True, test_mode=config.test_mode
        ).model

new_model = SegVol(
    model.image_encoder,
    model.mask_decoder,
    model.prompt_encoder,
    roi_size=[1],
    patch_size=[2],
    test_mode=False
)
new_model.feat_shape = model.feat_shape
new_model.text_encoder = TextEncoder()
new_model.load_state_dict(model.state_dict())

monai.networks.nets.vit ViT.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.


<All keys matched successfully>

In [7]:
# NOTE!! I'm drunk, so check this later
# The image looks currently looks like: (1, 32, 256, 256)
# However, the patch embedding expects (B, C, H, W, D)
image_bchwd = image.unsqueeze(0)

In [8]:
new_model

SegVol(
  (image_encoder): ViT(
    (patch_embedding): PatchEmbeddingBlock(
      (patch_embeddings): Sequential(
        (0): Rearrange('b c (h p1) (w p2) (d p3) -> b (h w d) (p1 p2 p3 c)', p1=4, p2=16, p3=16)
        (1): Linear(in_features=1024, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (blocks): ModuleList(
      (0-11): 12 x TransformerBlock(
        (mlp): MLPBlock(
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (linear2): Linear(in_features=3072, out_features=768, bias=True)
          (fn): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (drop2): Dropout(p=0.0, inplace=False)
        )
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SABlock(
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (qkv): Linear(in_features=768, out_features=2304, bias=False)
          (input_rearrange): Re

In [ ]:
organs_cls = ["esophagus", "heart", "trachea", "aorta"]
loss = new_model(image_bchwd, train_organs=organs_cls, train_labels=gt3D, modality="CT") 


Helo from SegVol
